In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv("ibd_source_data.csv")
df.head()

,AUTO_ID,PAT_ID,PAT_MRN_ID,MARITAL_STATUS,EMPLOYMENT_STATUS,GENDER,AGE,DISTANCE_KM,SIBDQ,ENC_CONSULT,...,ANTI_IL12,ANTI_INTEGRIN,ANTI_TNF,Immunomodulators,Systemic_steroids,Vitamin_D,ANTIBIOTICS,PSYCH_DX,TOBACCO_USE,CHARGE_FLAG_85pctile
0,0,156369,156369,Married,Employed,Male,Age3,Dist3,Good,Never,...,Never,Never,Never,Persistent,Persistent,Never,Never,NaN,Quit,0.0
1,1,174687646,841189757,Single,NotEmployed,Male,Age1,Dist1,Good,Never,...,Never,Never,Never,Never,Never,Never,Persistent,NaN,Yes,0.0
2,2,738090216,738090216,Married,NotEmployed,Female,Age1,Dist3,Good,Never,...,Never,Never,Never,Persistent,Never,Never,Transient,NaN,Never,0.0
3,3,176623065,841748158,Single,Employed,Female,Age2,Dist1,NaN,Never,...,Never,Never,Never,Never,Never,Transient,Transient,NaN,Never,0.0
4,4,60187,60187,Married,Employed,Female,Age3,Dist2,Bad,Never,...,Never,Never,Never,Never,Never,Never,Never,NaN,Never,0.0


In [3]:
def create_node(node_name):
    # {"node":0,"name":"node0"}
    
    global node_id_count
    node = {
        "node" : node_id_count,
        "name" : node_name
    }
    node_id_count = node_id_count + 1
    return node

def create_link(source_id, target_id, val):
    # {"source":0,"target":2,"value":2}
    global node_id_count
    
    link = {
        "source" : source_id,
        "target" : target_id,
        "value" : val
    }
    return link;
    

In [4]:
def f(q, sel_data_col, source_id):
    global nodes
    global links
    
    temp_dict = {}
    result = []
    
    if q == "":
        temp_dict = df[sel_data_col].value_counts().to_dict()
        node = create_node(sel_data_col)
        nodes.append(node)
    else:
        temp_dict = df.query(q)[sel_data_col].value_counts().to_dict()
    
    
    for key, val in temp_dict.items():
        node = create_node(key)
        nodes.append(node)
        result.append(node)
        
        link = create_link(source_id, node["node"], val)
        links.append(link)
        
    
    return result

In [5]:
def get_next_col_from_list(cur_col_name):
    global vars_list
    
    for i in range(0, len(vars_list)):
        if vars_list[i] == cur_col_name and i < (len(vars_list) - 1):
            return vars_list[i + 1]
        
    return ""

In [7]:
nodes = []
links = []
node_id_count = 0

#vars_list = ["GENDER","AGE","EMPLOYMENT_STATUS","Systemic_steroids","Vitamin_D"]
#vars_list = ["GENDER","AGE","Systemic_steroids"]
vars_list = ["GENDER","Systemic_steroids"]
#node_list = f("", 'GENDER',0)

def build_chart_data(prev_q, prev_col, cur_col, node_list):
    global var_list
    # Get next column name from the list
    next_col = get_next_col_from_list(cur_col)
    if next_col != "":
        if prev_q == "" and node_list == None:
            nl = f("", cur_col, 0) # Create new list of nodes
            build_chart_data("", cur_col, next_col, nl)
        else:
            for item in node_list:
                if prev_q == "":
                    q = prev_col + " == '" + item["name"] + "'"
                else:
                    q = prev_q + " & " + prev_col + " == '" + item["name"] + "'"

                nl = f(q, cur_col, int(item["node"]))
                build_chart_data(q, cur_col, next_col, nl)

#for item in node_list:
#    q = "GENDER=='" + item["name"] + "'"
#    col = "AGE"
#    print(item["node"])
#    f(q, col, int(item["node"]))

q = "GENDER=='Male'"
col = "AGE"
#    print(item["node"])
f(q, col, 0)

#build_chart_data("", "", "GENDER", None)


result = {
    "nodes" : nodes,
    "links" : links
}
print(json.dumps(result))



{"nodes": [{"node": 0, "name": "Age2"}, {"node": 1, "name": "Age3"}, {"node": 2, "name": "Age1"}, {"node": 3, "name": "Age4"}], "links": [{"source": 0, "target": 0, "value": 500}, {"source": 0, "target": 1, "value": 404}, {"source": 0, "target": 2, "value": 390}, {"source": 0, "target": 3, "value": 175}]}
